In [825]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

# EDA + baseline

Cчитаем датасет

In [826]:
tr = pd.read_csv("tr_for_students.csv")
tr

,issue_d,purpose,addr_state,sub_grade,home_ownership,emp_title,installment,dti,funded_amnt,annual_inc,...,mths_since_recent_inq,delinq_2yrs,chargeoff_within_12_mths,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def
0,2012-01-01,debt_consolidation,IL,C3,RENT,Chicago Shakespeare Theatre,172.48,16.39,5000,21600.0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0
1,2013-01-01,debt_consolidation,FL,B4,MORTGAGE,Queen of Pawns,674.94,16.31,20000,95000.0,...,4.0,0,0,0.0,0.0,4.0,23735.0,153379.0,0,0
2,2013-07-01,debt_consolidation,NJ,B5,RENT,tek services,809.24,14.01,24000,90000.0,...,19.0,0,0,0.0,0.0,5.0,4070.0,50834.0,0,0
3,2014-03-01,debt_consolidation,MI,C2,MORTGAGE,Inventory Control,342.56,14.51,10000,43000.0,...,5.0,1,1,4.0,1.0,3.0,1127.0,32683.0,0,0
4,2013-10-01,credit_card,TX,D2,MORTGAGE,Business Analyst,473.58,22.65,13000,62000.0,...,5.0,0,0,0.0,0.0,5.0,3443.0,59495.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61164,2013-02-01,debt_consolidation,CA,A5,RENT,Cupertino Union School District,583.47,7.96,18375,133325.0,...,NaN,1,0,1.0,1.0,1.0,5278.0,48557.0,0,0
61165,2013-07-01,debt_consolidation,TX,A5,MORTGAGE,Los Fresnos CISD,341.72,20.97,16500,82000.0,...,1.0,0,0,0.0,0.0,3.0,18594.0,333493.0,0,0
61166,2013-04-01,debt_consolidation,FL,D1,RENT,Publix,245.06,33.65,6800,27000.0,...,5.0,0,0,1.0,0.0,5.0,2093.0,36095.0,0,0
61167,2010-10-01,debt_consolidation,TX,B5,MORTGAGE,us postal,107.73,18.48,3250,58000.0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0


Посмотрим на распределение целевой переменной, а потом поработаем с пропусками

In [827]:
pie_df = pd.DataFrame(tr['def'].value_counts(normalize=True))
pie_df.reset_index(inplace=True)
pie_df['def'] = round(pie_df['def'], 2)
fig = px.pie(pie_df, values='def', names='index', color='index', color_discrete_map={1: '#9467BD', 0: '#FECB52'}, title='Non-Default (0) vs Default (1) Distribution')
fig.show()

Отметим, что распределение таргета не равномерно. Будем использовать эту информацию при разделении выборки на train и test

Теперь посмотрим на пропуски в данных

In [828]:
tr.isnull().sum()

issue_d                         0
purpose                         0
addr_state                      0
sub_grade                       0
home_ownership                  0
emp_title                    3865
installment                     0
dti                             0
funded_amnt                     0
annual_inc                      0
emp_length                   2674
term                            0
inq_last_6mths                  0
mths_since_recent_inq       13529
delinq_2yrs                     0
chargeoff_within_12_mths        0
num_accts_ever_120_pd       11941
num_tl_90g_dpd_24m          11941
acc_open_past_24mths         7886
avg_cur_bal                 11945
tot_hi_cred_lim             11941
delinq_amnt                     0
def                             0
dtype: int64

Видим, что колонки с количеством аккаунтов с задолженностями (>120 дней, >90 дней за последние 2 года), лимитом по кредиту и средним балансом по аккаунтам имеют практически одинаковое количество пропущенных значений. Проверим, одинаковые ли это строки

In [829]:
tr[tr['num_accts_ever_120_pd'].isnull()].isnull().sum()

issue_d                         0
purpose                         0
addr_state                      0
sub_grade                       0
home_ownership                  0
emp_title                     725
installment                     0
dti                             0
funded_amnt                     0
annual_inc                      0
emp_length                    414
term                            0
inq_last_6mths                  0
mths_since_recent_inq        8415
delinq_2yrs                     0
chargeoff_within_12_mths        0
num_accts_ever_120_pd       11941
num_tl_90g_dpd_24m          11941
acc_open_past_24mths         7886
avg_cur_bal                 11941
tot_hi_cred_lim             11941
delinq_amnt                     0
def                             0
dtype: int64

Видим, что это одни и те же люди. Возможно, это означает, что у человека до этого времени не было кредитов, поэтому данные недоступны. Заполним эти значения нулями и создадим колонку, которая будет говорить, было это значение заполнено или нет (0 – в колонках num_accts_ever_120_pd, num_tl_90g_dpd_24m, tot_hi_cred_lim не было NaN, 1 – был)

In [830]:
null_num_accts_ever_120_pd = tr[tr['num_accts_ever_120_pd'].isnull()].index

tr.loc[null_num_accts_ever_120_pd, 'num_accts_ever_120_pd'] = 0
tr.loc[null_num_accts_ever_120_pd, 'num_tl_90g_dpd_24m'] = 0
tr.loc[null_num_accts_ever_120_pd, 'avg_cur_bal'] = 0
tr.loc[null_num_accts_ever_120_pd, 'tot_hi_cred_lim'] = 0

tr.loc[null_num_accts_ever_120_pd, 'num_accts_ever_120_pd_changed_from_nan'] = 1
tr['num_accts_ever_120_pd_changed_from_nan'] = tr['num_accts_ever_120_pd_changed_from_nan'].fillna(0)

tr.loc[null_num_accts_ever_120_pd, 'acc_open_past_24mths'] = 0
tr.loc[null_num_accts_ever_120_pd, 'acc_open_past_24mths_changed_from_nan'] = 1
tr['acc_open_past_24mths_changed_from_nan'] = tr['acc_open_past_24mths_changed_from_nan'].fillna(0)

tr

,issue_d,purpose,addr_state,sub_grade,home_ownership,emp_title,installment,dti,funded_amnt,annual_inc,...,chargeoff_within_12_mths,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,num_accts_ever_120_pd_changed_from_nan,acc_open_past_24mths_changed_from_nan
0,2012-01-01,debt_consolidation,IL,C3,RENT,Chicago Shakespeare Theatre,172.48,16.39,5000,21600.0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,1.0,1.0
1,2013-01-01,debt_consolidation,FL,B4,MORTGAGE,Queen of Pawns,674.94,16.31,20000,95000.0,...,0,0.0,0.0,4.0,23735.0,153379.0,0,0,0.0,0.0
2,2013-07-01,debt_consolidation,NJ,B5,RENT,tek services,809.24,14.01,24000,90000.0,...,0,0.0,0.0,5.0,4070.0,50834.0,0,0,0.0,0.0
3,2014-03-01,debt_consolidation,MI,C2,MORTGAGE,Inventory Control,342.56,14.51,10000,43000.0,...,1,4.0,1.0,3.0,1127.0,32683.0,0,0,0.0,0.0
4,2013-10-01,credit_card,TX,D2,MORTGAGE,Business Analyst,473.58,22.65,13000,62000.0,...,0,0.0,0.0,5.0,3443.0,59495.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61164,2013-02-01,debt_consolidation,CA,A5,RENT,Cupertino Union School District,583.47,7.96,18375,133325.0,...,0,1.0,1.0,1.0,5278.0,48557.0,0,0,0.0,0.0
61165,2013-07-01,debt_consolidation,TX,A5,MORTGAGE,Los Fresnos CISD,341.72,20.97,16500,82000.0,...,0,0.0,0.0,3.0,18594.0,333493.0,0,0,0.0,0.0
61166,2013-04-01,debt_consolidation,FL,D1,RENT,Publix,245.06,33.65,6800,27000.0,...,0,1.0,0.0,5.0,2093.0,36095.0,0,0,0.0,0.0
61167,2010-10-01,debt_consolidation,TX,B5,MORTGAGE,us postal,107.73,18.48,3250,58000.0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,1.0,1.0


Если нет запросов на кредиты за последние 6 месяцев и неизвестно время с последнего запроса, будем считать, что человек не делал запросов совсем (но сохраним информацию об этом в доп. колонку)

In [831]:
tr.loc[tr[tr[tr['inq_last_6mths'] == 0].isnull()].index, 'mths_since_recent_inq'] = 0
tr.loc[tr[tr[tr['inq_last_6mths'] == 0].isnull()].index, 'mths_since_recent_inq_changed_from_nan'] = 1
tr['mths_since_recent_inq_changed_from_nan'] = tr['mths_since_recent_inq_changed_from_nan'].fillna(0)
tr

,issue_d,purpose,addr_state,sub_grade,home_ownership,emp_title,installment,dti,funded_amnt,annual_inc,...,num_accts_ever_120_pd,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,num_accts_ever_120_pd_changed_from_nan,acc_open_past_24mths_changed_from_nan,mths_since_recent_inq_changed_from_nan
0,2012-01-01,debt_consolidation,IL,C3,RENT,Chicago Shakespeare Theatre,172.48,16.39,5000,21600.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1.0,1.0,1.0
1,2013-01-01,debt_consolidation,FL,B4,MORTGAGE,Queen of Pawns,674.94,16.31,20000,95000.0,...,0.0,0.0,4.0,23735.0,153379.0,0,0,0.0,0.0,1.0
2,2013-07-01,debt_consolidation,NJ,B5,RENT,tek services,809.24,14.01,24000,90000.0,...,0.0,0.0,5.0,4070.0,50834.0,0,0,0.0,0.0,1.0
3,2014-03-01,debt_consolidation,MI,C2,MORTGAGE,Inventory Control,342.56,14.51,10000,43000.0,...,4.0,1.0,3.0,1127.0,32683.0,0,0,0.0,0.0,1.0
4,2013-10-01,credit_card,TX,D2,MORTGAGE,Business Analyst,473.58,22.65,13000,62000.0,...,0.0,0.0,5.0,3443.0,59495.0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61164,2013-02-01,debt_consolidation,CA,A5,RENT,Cupertino Union School District,583.47,7.96,18375,133325.0,...,1.0,1.0,1.0,5278.0,48557.0,0,0,0.0,0.0,1.0
61165,2013-07-01,debt_consolidation,TX,A5,MORTGAGE,Los Fresnos CISD,341.72,20.97,16500,82000.0,...,0.0,0.0,3.0,18594.0,333493.0,0,0,0.0,0.0,1.0
61166,2013-04-01,debt_consolidation,FL,D1,RENT,Publix,245.06,33.65,6800,27000.0,...,1.0,0.0,5.0,2093.0,36095.0,0,0,0.0,0.0,1.0
61167,2010-10-01,debt_consolidation,TX,B5,MORTGAGE,us postal,107.73,18.48,3250,58000.0,...,0.0,0.0,0.0,0.0,0.0,0,0,1.0,1.0,1.0


In [832]:
tr.isnull().sum()

issue_d                                      0
purpose                                      0
addr_state                                   0
sub_grade                                    0
home_ownership                               0
emp_title                                 3865
installment                                  0
dti                                          0
funded_amnt                                  0
annual_inc                                   0
emp_length                                2674
term                                         0
inq_last_6mths                               0
mths_since_recent_inq                        0
delinq_2yrs                                  0
chargeoff_within_12_mths                     0
num_accts_ever_120_pd                        0
num_tl_90g_dpd_24m                           0
acc_open_past_24mths                         0
avg_cur_bal                                  4
tot_hi_cred_lim                              0
delinq_amnt  

Поработаем с названием должности

In [833]:
tr['employment_known'] = tr['emp_title'].apply(lambda x: 0 if pd.isna(x) else 1)
tr.loc[tr[tr['emp_title'].isnull()].index, 'emp_title'] = 'Unknown'
tr['emp_length'] = tr['emp_title'].apply(lambda x: len(x) if x != 'Unknown' else 0)
tr

,issue_d,purpose,addr_state,sub_grade,home_ownership,emp_title,installment,dti,funded_amnt,annual_inc,...,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,num_accts_ever_120_pd_changed_from_nan,acc_open_past_24mths_changed_from_nan,mths_since_recent_inq_changed_from_nan,employment_known
0,2012-01-01,debt_consolidation,IL,C3,RENT,Chicago Shakespeare Theatre,172.48,16.39,5000,21600.0,...,0.0,0.0,0.0,0.0,0,0,1.0,1.0,1.0,1
1,2013-01-01,debt_consolidation,FL,B4,MORTGAGE,Queen of Pawns,674.94,16.31,20000,95000.0,...,0.0,4.0,23735.0,153379.0,0,0,0.0,0.0,1.0,1
2,2013-07-01,debt_consolidation,NJ,B5,RENT,tek services,809.24,14.01,24000,90000.0,...,0.0,5.0,4070.0,50834.0,0,0,0.0,0.0,1.0,1
3,2014-03-01,debt_consolidation,MI,C2,MORTGAGE,Inventory Control,342.56,14.51,10000,43000.0,...,1.0,3.0,1127.0,32683.0,0,0,0.0,0.0,1.0,1
4,2013-10-01,credit_card,TX,D2,MORTGAGE,Business Analyst,473.58,22.65,13000,62000.0,...,0.0,5.0,3443.0,59495.0,0,0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61164,2013-02-01,debt_consolidation,CA,A5,RENT,Cupertino Union School District,583.47,7.96,18375,133325.0,...,1.0,1.0,5278.0,48557.0,0,0,0.0,0.0,1.0,1
61165,2013-07-01,debt_consolidation,TX,A5,MORTGAGE,Los Fresnos CISD,341.72,20.97,16500,82000.0,...,0.0,3.0,18594.0,333493.0,0,0,0.0,0.0,1.0,1
61166,2013-04-01,debt_consolidation,FL,D1,RENT,Publix,245.06,33.65,6800,27000.0,...,0.0,5.0,2093.0,36095.0,0,0,0.0,0.0,1.0,1
61167,2010-10-01,debt_consolidation,TX,B5,MORTGAGE,us postal,107.73,18.48,3250,58000.0,...,0.0,0.0,0.0,0.0,0,0,1.0,1.0,1.0,1


In [834]:
tr.isnull().sum()

issue_d                                   0
purpose                                   0
addr_state                                0
sub_grade                                 0
home_ownership                            0
emp_title                                 0
installment                               0
dti                                       0
funded_amnt                               0
annual_inc                                0
emp_length                                0
term                                      0
inq_last_6mths                            0
mths_since_recent_inq                     0
delinq_2yrs                               0
chargeoff_within_12_mths                  0
num_accts_ever_120_pd                     0
num_tl_90g_dpd_24m                        0
acc_open_past_24mths                      0
avg_cur_bal                               4
tot_hi_cred_lim                           0
delinq_amnt                               0
def                             

avg_cur_bal заменим медианным значением (не так критично, потому что неизвестных значений всего 4 из >60000)

In [835]:
tr.loc[tr[tr['avg_cur_bal'].isnull()].index, 'avg_cur_bal'] = tr[tr['avg_cur_bal'].isnull() == False]['avg_cur_bal'].median()
tr

,issue_d,purpose,addr_state,sub_grade,home_ownership,emp_title,installment,dti,funded_amnt,annual_inc,...,num_tl_90g_dpd_24m,acc_open_past_24mths,avg_cur_bal,tot_hi_cred_lim,delinq_amnt,def,num_accts_ever_120_pd_changed_from_nan,acc_open_past_24mths_changed_from_nan,mths_since_recent_inq_changed_from_nan,employment_known
0,2012-01-01,debt_consolidation,IL,C3,RENT,Chicago Shakespeare Theatre,172.48,16.39,5000,21600.0,...,0.0,0.0,0.0,0.0,0,0,1.0,1.0,1.0,1
1,2013-01-01,debt_consolidation,FL,B4,MORTGAGE,Queen of Pawns,674.94,16.31,20000,95000.0,...,0.0,4.0,23735.0,153379.0,0,0,0.0,0.0,1.0,1
2,2013-07-01,debt_consolidation,NJ,B5,RENT,tek services,809.24,14.01,24000,90000.0,...,0.0,5.0,4070.0,50834.0,0,0,0.0,0.0,1.0,1
3,2014-03-01,debt_consolidation,MI,C2,MORTGAGE,Inventory Control,342.56,14.51,10000,43000.0,...,1.0,3.0,1127.0,32683.0,0,0,0.0,0.0,1.0,1
4,2013-10-01,credit_card,TX,D2,MORTGAGE,Business Analyst,473.58,22.65,13000,62000.0,...,0.0,5.0,3443.0,59495.0,0,0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61164,2013-02-01,debt_consolidation,CA,A5,RENT,Cupertino Union School District,583.47,7.96,18375,133325.0,...,1.0,1.0,5278.0,48557.0,0,0,0.0,0.0,1.0,1
61165,2013-07-01,debt_consolidation,TX,A5,MORTGAGE,Los Fresnos CISD,341.72,20.97,16500,82000.0,...,0.0,3.0,18594.0,333493.0,0,0,0.0,0.0,1.0,1
61166,2013-04-01,debt_consolidation,FL,D1,RENT,Publix,245.06,33.65,6800,27000.0,...,0.0,5.0,2093.0,36095.0,0,0,0.0,0.0,1.0,1
61167,2010-10-01,debt_consolidation,TX,B5,MORTGAGE,us postal,107.73,18.48,3250,58000.0,...,0.0,0.0,0.0,0.0,0,0,1.0,1.0,1.0,1


Теперь создадим несколько новых признаков. Начнем с разбиения даты (issue_d) на год, месяц и день недели

In [836]:
dates = tr['issue_d'].astype(np.datetime64)
tr['issue_year'] = dates.apply(lambda x: x.year)
tr['issue_month'] = dates.apply(lambda x: x.month)
tr['issue_weekday'] = dates.apply(lambda x: x.weekday())
# все подавали заявку 1 числа, поэтому этот признак не нужен
tr = tr.drop(columns='issue_d', axis = 1)
tr

,purpose,addr_state,sub_grade,home_ownership,emp_title,installment,dti,funded_amnt,annual_inc,emp_length,...,tot_hi_cred_lim,delinq_amnt,def,num_accts_ever_120_pd_changed_from_nan,acc_open_past_24mths_changed_from_nan,mths_since_recent_inq_changed_from_nan,employment_known,issue_year,issue_month,issue_weekday
0,debt_consolidation,IL,C3,RENT,Chicago Shakespeare Theatre,172.48,16.39,5000,21600.0,27,...,0.0,0,0,1.0,1.0,1.0,1,2012,1,6
1,debt_consolidation,FL,B4,MORTGAGE,Queen of Pawns,674.94,16.31,20000,95000.0,14,...,153379.0,0,0,0.0,0.0,1.0,1,2013,1,1
2,debt_consolidation,NJ,B5,RENT,tek services,809.24,14.01,24000,90000.0,12,...,50834.0,0,0,0.0,0.0,1.0,1,2013,7,0
3,debt_consolidation,MI,C2,MORTGAGE,Inventory Control,342.56,14.51,10000,43000.0,17,...,32683.0,0,0,0.0,0.0,1.0,1,2014,3,5
4,credit_card,TX,D2,MORTGAGE,Business Analyst,473.58,22.65,13000,62000.0,16,...,59495.0,0,0,0.0,0.0,1.0,1,2013,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61164,debt_consolidation,CA,A5,RENT,Cupertino Union School District,583.47,7.96,18375,133325.0,31,...,48557.0,0,0,0.0,0.0,1.0,1,2013,2,4
61165,debt_consolidation,TX,A5,MORTGAGE,Los Fresnos CISD,341.72,20.97,16500,82000.0,16,...,333493.0,0,0,0.0,0.0,1.0,1,2013,7,0
61166,debt_consolidation,FL,D1,RENT,Publix,245.06,33.65,6800,27000.0,6,...,36095.0,0,0,0.0,0.0,1.0,1,2013,4,0
61167,debt_consolidation,TX,B5,MORTGAGE,us postal,107.73,18.48,3250,58000.0,9,...,0.0,0,0,1.0,1.0,1.0,1,2010,10,4


Далее посмотрим на распределения таргета в зависимости от значения sub_grade и созданного при помощи этой колонки значения признака grade (более общий). Для этого создадим функцию, которая будет рисовать нам красивое распределение таргета в зависимости от значений заданного признака, и назовем ее build_default_distribution(feature)

In [837]:
def build_default_distribution(feature):
    column_def = tr[tr['def'] == 1][feature].value_counts()
    column_non = tr[tr['def'] == 0][feature].value_counts()
    df = pd.DataFrame({'default' : column_def, 'non' : column_non})
    df['def_%'] = round(df['default'] / (df['default'] + df['non']), 2)
    df['non_%'] = round(df['non'] / (df['default'] + df['non']), 2)
    df = df.drop(columns = ['default', 'non'])
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index' : feature})
    
    df1 = df.loc[df.index, [feature, 'def_%']]
    df1['class'] = ['default'] * len(df1)
    df1 = df1.rename(columns = {'def_%' : 'proportion'})
    
    df2 = df.loc[df.index, [feature, 'non_%']]
    df2['class'] = ['non-default'] * len(df2)
    df2 = df2.rename(columns = {'non_%' : 'proportion'})
    
    df_new = pd.concat([df1, df2])
    fig = px.bar(df_new, x=feature, y="proportion", color="class", color_discrete_map={'default': '#9467BD', 'non-default': '#FECB52'}, text_auto=True)
    fig.update_layout(title = "Default / Non-default Distribution by " + feature)
    fig.show()

In [838]:
tr['grade'] = tr['sub_grade'].apply(lambda x : x[0])
tr['grade'].value_counts()

B    19181
C    16243
A     9746
D     9516
E     4336
F     1747
G      400
Name: grade, dtype: int64

In [839]:
build_default_distribution("grade")
build_default_distribution("sub_grade")

Будем иметь в виду, что в будущем будет использоваться WOE-преобразование признаков, поэтому сохраним признак grade вместо sub_grade, так как по нему зависимость таргета видна более четко

Значений у колонки с штатом очень много, при этом не все они репрезентативны, так как соответствуют всего нескольким потенциальным заемщикам

In [840]:
tr['addr_state'].value_counts(normalize=True)

CA    0.158397
NY    0.084798
TX    0.075512
FL    0.067403
IL    0.038582
NJ    0.038336
PA    0.034331
GA    0.032860
OH    0.032124
VA    0.031127
NC    0.028854
MI    0.024604
WA    0.023721
MD    0.023525
MA    0.023345
AZ    0.022462
CO    0.021432
MN    0.017509
MO    0.017443
CT    0.015809
NV    0.014615
OR    0.013520
IN    0.013209
AL    0.012719
WI    0.012572
LA    0.011574
TN    0.011395
SC    0.011166
KY    0.009433
KS    0.009269
OK    0.008305
AR    0.007684
UT    0.007373
NM    0.005673
WV    0.005591
HI    0.005182
NH    0.004872
RI    0.004201
MT    0.003237
DC    0.003172
AK    0.003155
DE    0.002877
SD    0.002403
WY    0.002191
VT    0.001962
MS    0.000458
IA    0.000016
Name: addr_state, dtype: float64

In [841]:
freq_states = tr['addr_state'].value_counts(normalize=True)[:10]
tr['addr_state'] = tr['addr_state'].apply(lambda x: x if x in freq_states else 'Rare')

Посмотрим на описательные статистики

In [842]:
tr.describe()

,installment,dti,funded_amnt,annual_inc,emp_length,term,inq_last_6mths,mths_since_recent_inq,delinq_2yrs,chargeoff_within_12_mths,...,tot_hi_cred_lim,delinq_amnt,def,num_accts_ever_120_pd_changed_from_nan,acc_open_past_24mths_changed_from_nan,mths_since_recent_inq_changed_from_nan,employment_known,issue_year,issue_month,issue_weekday
count,61169.000000,61169.000000,61169.000000,6.116900e+04,61169.000000,61169.000000,61169.000000,61169.0,61169.000000,61169.000000,...,6.116900e+04,61169.000000,61169.000000,61169.000000,61169.000000,61169.0,61169.000000,61169.000000,61169.000000,61169.000000
mean,432.671268,16.782808,14159.664618,7.271755e+04,16.119456,42.276120,0.831254,0.0,0.259069,0.006458,...,1.345247e+05,12.090634,0.159591,0.195213,0.195213,1.0,0.936814,2012.864343,6.202178,3.170675
std,243.699189,7.531950,8181.125909,4.954698e+04,9.048074,10.546991,1.072571,0.0,0.761321,0.089726,...,1.662111e+05,720.055058,0.366229,0.396368,0.396368,0.0,0.243298,1.009278,3.479398,1.943848
min,4.930000,0.000000,1000.000000,6.695000e+03,0.000000,36.000000,0.000000,0.0,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,2010.000000,1.000000,0.000000
25%,257.540000,11.180000,8000.000000,4.500000e+04,9.000000,36.000000,0.000000,0.0,0.000000,0.000000,...,2.000000e+04,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,2012.000000,3.000000,1.000000
50%,382.490000,16.500000,12000.000000,6.250000e+04,16.000000,36.000000,0.000000,0.0,0.000000,0.000000,...,6.897300e+04,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,2013.000000,6.000000,3.000000
75%,565.310000,22.200000,20000.000000,8.800000e+04,22.000000,60.000000,1.000000,0.0,0.000000,0.000000,...,2.117930e+05,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,2014.000000,9.000000,5.000000
max,1408.130000,34.990000,35000.000000,3.900000e+06,64.000000,60.000000,8.000000,0.0,29.000000,3.000000,...,9.999999e+06,70076.000000,1.000000,1.000000,1.000000,1.0,1.000000,2014.000000,12.000000,6.000000


Теперь удалим ненужные колонки и заменим некоторые из них на признаки-индикаторы (удалим один из них в каждой категории, чтобы не было линейной зависимости)

In [843]:
tr = tr.drop(columns = ['sub_grade', 'emp_title'])

In [844]:
tr = pd.get_dummies(tr, columns=['purpose', 'addr_state', 'grade', 'home_ownership', 'term'], drop_first = True)

Теперь обучим обычную логистическую регрессию без WOE-преобразования. Это будет наш baseline

In [846]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tr.drop(columns = "def"), tr["def"], test_size=0.25, stratify = tr["def"], random_state=42)

In [847]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
lr = LogisticRegression(random_state = 42)
lr.fit(X_train, y_train)
preds = lr.predict_proba(X_test)
print(roc_auc_score(y_test, preds[:, 1]))

0.6180065544356939
